In [2]:
import sys
sys.path.append("../")


import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
from openpyxl import load_workbook
import engarde.decorators as ed

from library import test
@ed.verify(test.alldois)
@ed.verify(test.allyearsanddistricts)
def load(data):
    return data

In [16]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/impact/'
data = pd.read_csv(os.path.join(data_path, 'tea', 'desc_yr1718.csv'),
                  sep=",", low_memory=False)
data[data.district == 250907]['distname']

1191    WINNSBORO ISD
Name: distname, dtype: object

In [27]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/impact/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'master_data.csv'),
                  sep=",", low_memory=False)
load(data)

,Unnamed: 0,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,type,...,type_urban,type_suburban,type_town,type_rural,index,always_eligible,teachers_nodegree,teachers_badegree,teachers_msdegree,teachers_phddegree
0,0,1902,CAYUGA ISD,2012,ANDERSON,N,NaN,NaN,NaN,H,...,0,0,0,1,186,False,0.000000,0.457439,0.062117,0.000000
1,1,1903,ELKHART ISD,2012,ANDERSON,N,NaN,NaN,NaN,G,...,0,0,1,0,355,False,0.000000,0.523077,0.108609,0.000000
2,2,1904,FRANKSTON ISD,2012,ANDERSON,N,NaN,NaN,NaN,H,...,0,0,0,1,400,False,0.000000,0.452648,0.095202,0.000000
3,3,1906,NECHES ISD,2012,ANDERSON,N,NaN,NaN,NaN,H,...,0,0,0,1,805,False,0.000000,0.553633,0.017301,0.000000
4,4,1907,PALESTINE ISD,2012,ANDERSON,N,NaN,NaN,NaN,E,...,0,0,1,0,859,False,0.002184,0.439686,0.099342,0.002316
5,5,1908,WESTWOOD ISD,2012,ANDERSON,N,NaN,NaN,NaN,G,...,0,0,1,0,1249,False,0.000000,0.468405,0.093822,0.004691
6,6,1909,SLOCUM ISD,2012,ANDERSON,N,NaN,NaN,NaN,H,...,0,0,0,1,1064,False,0.000000,0.618001,0.025764,0.000000
7,7,2901,ANDREWS ISD,2012,ANDREWS,N,NaN,NaN,NaN,D,...,0,1,0,0,38,False,0.000000,0.434661,0.086919,0.000000
8,8,3801,PINEYWOODS COMMUNITY ACADEMY,2012,ANGELINA,Y,NaN,NaN,NaN,I,...,0,0,1,0,890,False,0.000000,0.399410,0.154429,0.013407
9,9,3902,HUDSON ISD,2012,ANGELINA,N,NaN,NaN,NaN,G,...,0,0,1,0,542,True,0.005473,0.408490,0.066222,0.000000


In [28]:
data[data.doi == True].distname.nunique()

824

In [10]:
def allyearsanddistricts(data, just_years = [2012, 2013, 2014, 2015, 2016, 2017, 2018]):
    ground_truth = {2012: 1227,
                    2013: 1228,
                    2014: 1227,
                    2015: 1219,
                    2016: 1207,
                    2017: 1203,
                    2018: 1200}
    count_correct = 0
    count_total = len(just_years)
    for yr in just_years:
        if len(data[data.year == yr]) == ground_truth[yr]:
            count_correct = count_correct + 1
        if len(data[data.year == yr]) != ground_truth[yr]:
            print('Year ', yr, 'should have', ground_truth[yr], 'districts. But has ', len(data[data.year == yr]))
    correct = (count_correct == count_total)
    return correct
allyearsanddistricts(data)

True

In [11]:
cols = [c for c in data.columns if c.lower()[:3] != 'reg']
data = data[cols]

In [12]:
data[['distname', 'doi', 'doi_year']][data.distname == 'HUDSON ISD'].head(50)

,distname,doi,doi_year
9,HUDSON ISD,True,2017.0
1236,HUDSON ISD,True,2017.0
2464,HUDSON ISD,True,2017.0
3691,HUDSON ISD,True,2017.0
4910,HUDSON ISD,True,2017.0
6117,HUDSON ISD,True,2017.0
7320,HUDSON ISD,True,2017.0


In [13]:
import sys
sys.path.append("../..")

import os
import pandas as pd
import numpy as np
from merge_and_clean.library import start
from merge_and_clean.library import clean_for_merge

tea = pd.read_csv(os.path.join(start.data_path, 'tea', 'desc_long.csv'),
                  sep=",", low_memory = False)
tea = tea[['district', 'distname', 'year',
           'cntyname', 'distischarter', 'rating_academic', 'rating_financial', 'eligible',
           'type', 'type_description',
           'schools_num',
           'students_num', 'students_frpl_num',
           'students_black_num', 'students_hisp_num', 'students_white_num',
           'students_amind_num', 'students_asian_num', 'students_paci_num', 'students_tworaces_num',
           'teachers_num', 'teachers_new_num', 'teachers_turnover_num',
           'teachers_turnover_denom', 'teachers_turnover_ratio',
           'teachers_exp_ave', 'teachers_tenure_ave',
           'teachers_nodegree_num', 'teachers_badegree_num',
           'teachers_msdegree_num', 'teachers_phddegree_num',
           'r_3rd_avescore', 'r_3rd_numtakers',
           'm_3rd_avescore', 'm_3rd_numtakers',
           'r_4th_avescore', 'r_4th_numtakers',
           'm_4th_avescore', 'm_4th_numtakers',
           'r_5th_avescore', 'r_5th_numtakers',
           'm_5th_avescore', 'm_5th_numtakers',
           'r_6th_avescore', 'r_6th_numtakers',
           'm_6th_avescore', 'm_6th_numtakers',
           'r_7th_avescore', 'r_7th_numtakers',
           'm_7th_avescore', 'm_7th_numtakers',
           'r_8th_avescore', 'r_8th_numtakers',
           'm_8th_avescore', 'm_8th_numtakers',
           'alg_avescore', 'alg_numtakers',
           'bio_avescore', 'bio_numtakers',
           'eng1_avescore', 'eng1_numtakers',
           'eng2_avescore', 'eng2_numtakers',
           'us_avescore', 'us_numtakers']]

laws = pd.read_csv(os.path.join(start.data_path, 'plans', 'doi_final.csv'),
                   sep=",")
cols = [c for c in laws.columns if c.lower()[:7] != 'Unnamed']
laws = laws[cols]
laws = laws.rename({'district': 'distname'}, axis=1)
laws.head()

# Geographic data
geo = pd.read_csv(os.path.join(start.data_path, 'geo', '2016_txpopest_county.csv'),
                  sep=",")
geo = geo[['county', 'july1_2016_pop_est']]
geo = geo.rename({'july1_2016_pop_est': 'cnty_pop'}, axis='columns')
geo['cnty_pop'] = geo['cnty_pop'] / 1000
geo['cnty_pop'] = geo['cnty_pop'].round(0)
geo = clean_for_merge.uppercase_column(geo, 'county')

# # Clean variables for merge

# problems with district name from scraping
tea = tea.pipe(clean_for_merge.resolve_unicode_problems, 'distname')
laws = laws.pipe(clean_for_merge.resolve_unicode_problems, 'distname')

# scraped names in title case, but tea all caps. change scraped distname to caps
laws = laws.pipe(clean_for_merge.uppercase_column, 'distname')

# Add district numbers to some plans
laws = clean_for_merge.add_distnum_to_plan(laws, 'distname')

# sometimes districts named CISD othertimes ISD. Make all ISD
tea = clean_for_merge.replace_column_values(tea, 'distname', 'CISD', 'ISD')
laws = clean_for_merge.replace_column_values(laws, 'distname', 'CISD', 'ISD')

# fix district names that don't match
tea = clean_for_merge.sync_district_names(tea, 'distname')
laws = clean_for_merge.sync_district_names(laws, 'distname')

mismatch = clean_for_merge.get_not_in(laws, 'distname', tea, 'distname')
mismatch_list = clean_for_merge.strip_distnum_parens(list(mismatch.distname))

df = clean_for_merge.distnum_in_paren(
    tea[[elem in mismatch_list for elem in tea.distname]])

tea.loc[(tea['distname'].isin(mismatch_list)), 'distname'] = (
    tea.loc[(tea['distname'].isin(mismatch_list))]
        .pipe(clean_for_merge.distnum_in_paren)['distname']
)

../../merge_and_clean/library/clean_for_merge.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.pipe(lpad_nums, distnum, 6)[distnum].astype(str) + ')')


In [14]:
len(laws)

824

In [15]:
tea.year.value_counts()

2013    1228
2014    1227
2012    1227
2015    1219
2016    1207
2017    1203
2018    1200
Name: year, dtype: int64

In [16]:
tea.district.nunique()

1261

In [17]:
len(laws)

824

In [18]:
# # Merge
data = tea.merge(laws, left_on='distname', right_on='distname', how='outer', indicator=True)
data.loc[(data['_merge'] == 'both'), 'doi'] = True
data.loc[(data['_merge'] == 'left_only'), 'doi'] = False

In [22]:
load(data)

Year  2012 should have 1029 districts. But has  1227
Year  2013 should have 1026 districts. But has  1228
Year  2014 should have 1025 districts. But has  1227
Year  2015 should have 1024 districts. But has  1219
Year  2016 should have 1024 districts. But has  1207
Year  2017 should have 1023 districts. But has  1203
Year  2018 should have 1023 districts. But has  1200


AssertionError: ('allyearsandtpsd is not true',       district                            distname  year    cntyname  \
0         1902                          CAYUGA ISD  2012    ANDERSON   
1         1902                          CAYUGA ISD  2013    ANDERSON   
2         1902                          CAYUGA ISD  2014    ANDERSON   
3         1902                          CAYUGA ISD  2015    ANDERSON   
4         1902                          CAYUGA ISD  2016    ANDERSON   
5         1902                          CAYUGA ISD  2017    ANDERSON   
6         1902                          CAYUGA ISD  2018    ANDERSON   
7         1903                         ELKHART ISD  2012    ANDERSON   
8         1903                         ELKHART ISD  2013    ANDERSON   
9         1903                         ELKHART ISD  2014    ANDERSON   
10        1903                         ELKHART ISD  2015    ANDERSON   
11        1903                         ELKHART ISD  2016    ANDERSON   
12        1903  

In [19]:
data[data._merge == 'right_only']

,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,type,type_description,...,reg551_001,reg552_023,reg552_024,reg7_056,reg75_600,reg921_406,reg97_1001,reg97_1055,_merge,doi


In [20]:
data[data.distname.str.startswith("RICE")]

,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,type,type_description,...,reg551_001,reg552_023,reg552_024,reg7_056,reg75_600,reg921_406,reg97_1001,reg97_1055,_merge,doi
1478,45903,RICE ISD (045903),2012,COLORADO,N,NaN,NaN,NaN,G,NON-METROPOLITAN STABLE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
1479,45903,RICE ISD (045903),2013,COLORADO,N,M,NaN,NaN,G,NON-METROPOLITAN STABLE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
1480,45903,RICE ISD (045903),2014,COLORADO,N,M,NaN,NaN,G,NON-METROPOLITAN STABLE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
1481,45903,RICE ISD (045903),2015,COLORADO,N,M,Pass,True,G,NON-METROPOLITAN STABLE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
1482,45903,RICE ISD (045903),2016,COLORADO,N,M,Pass,True,G,NON-METROPOLITAN STABLE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
1483,45903,RICE ISD (045903),2017,COLORADO,N,M,Pass,True,G,NON-METROPOLITAN STABLE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
1484,45903,RICE ISD (045903),2018,COLORADO,N,Not Rated: Harvey Provision,Pass,False,G,NON-METROPOLITAN STABLE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
5961,175911,RICE ISD (175911),2012,NAVARRO,N,NaN,NaN,NaN,D,OTHER CENTRAL CITY SUBURBAN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
5962,175911,RICE ISD (175911),2013,NAVARRO,N,M,NaN,NaN,D,OTHER CENTRAL CITY SUBURBAN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True
5963,175911,RICE ISD (175911),2014,NAVARRO,N,M,NaN,NaN,D,OTHER CENTRAL CITY SUBURBAN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both,True


In [23]:
data[data.doi == True].district.nunique()

824

In [6]:
data[data.duplicated(subset = ['distname', 'year'])]

,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,type,type_description,...,reg55_0811,reg551_001,reg552_023,reg552_024,reg7_056,reg75_600,reg921_406,reg97_1001,reg97_1055,_merge


In [20]:
data[data._merge != 'both']

,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,type,type_description,...,reg55_0811,reg551_001,reg552_023,reg552_024,reg7_056,reg75_600,reg921_406,reg97_1001,reg97_1055,_merge
8,3801,PINEYWOODS COMMUNITY ACADEMY,2012,ANGELINA,Y,NaN,NaN,NaN,I,CHARTER SCHOOL DISTRICTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
12,3905,DIBOLL ISD,2012,ANGELINA,N,NaN,NaN,NaN,G,NON-METROPOLITAN STABLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
14,3907,CENTRAL ISD,2012,ANGELINA,N,NaN,NaN,NaN,G,NON-METROPOLITAN STABLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
20,7901,CHARLOTTE ISD,2012,ATASCOSA,N,NaN,NaN,NaN,H,RURAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
23,7905,PLEASANTON ISD,2012,ATASCOSA,N,NaN,NaN,NaN,E,INDEPENDENT TOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
24,7906,POTEET ISD,2012,ATASCOSA,N,NaN,NaN,NaN,G,NON-METROPOLITAN STABLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
25,8901,BELLVILLE ISD,2012,AUSTIN,N,NaN,NaN,NaN,E,INDEPENDENT TOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
26,8902,SEALY ISD,2012,AUSTIN,N,NaN,NaN,NaN,D,OTHER CENTRAL CITY SUBURBAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29,10901,MEDINA ISD,2012,BANDERA,N,NaN,NaN,NaN,H,RURAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
30,10902,BANDERA ISD,2012,BANDERA,N,NaN,NaN,NaN,B,MAJOR SUBURBAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [7]:
data.year.value_counts()

2013    1228
2014    1227
2012    1227
2015    1219
2016    1207
2017    1203
2018    1200
Name: year, dtype: int64

In [11]:
data[data._merge == 'right_only']

,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,type,type_description,...,reg55_0811,reg551_001,reg552_023,reg552_024,reg7_056,reg75_600,reg921_406,reg97_1001,reg97_1055,_merge
8528,NaN,HIGHLAND PARK ISD (7911),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,right_only


In [6]:
data[(data._merge == 'left_only') & (data.distname.str.startswith('HIGH'))]

,district,distname,year,cntyname,distischarter,rating_academic,rating_financial,eligible,type,type_description,...,reg55_0811,reg551_001,reg552_023,reg552_024,reg7_056,reg75_600,reg921_406,reg97_1001,reg97_1055,_merge
1981,57911.0,HIGHLAND PARK ISD (57911),2012.0,DALLAS,N,NaN,NaN,NaN,B,MAJOR SUBURBAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1982,57911.0,HIGHLAND PARK ISD (57911),2013.0,DALLAS,N,M,NaN,NaN,B,MAJOR SUBURBAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1983,57911.0,HIGHLAND PARK ISD (57911),2014.0,DALLAS,N,M,NaN,NaN,B,MAJOR SUBURBAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1984,57911.0,HIGHLAND PARK ISD (57911),2015.0,DALLAS,N,M,Pass,True,B,MAJOR SUBURBAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1985,57911.0,HIGHLAND PARK ISD (57911),2016.0,DALLAS,N,M,Pass,True,B,MAJOR SUBURBAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1986,57911.0,HIGHLAND PARK ISD (57911),2017.0,DALLAS,N,M,Pass,True,B,MAJOR SUBURBAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1987,57911.0,HIGHLAND PARK ISD (57911),2018.0,DALLAS,N,M,Pass,True,B,MAJOR SUBURBAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2869,84903.0,HIGH ISLAND ISD,2012.0,GALVESTON,N,NaN,NaN,NaN,H,RURAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2870,84903.0,HIGH ISLAND ISD,2013.0,GALVESTON,N,M,NaN,NaN,H,RURAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2871,84903.0,HIGH ISLAND ISD,2014.0,GALVESTON,N,I,NaN,NaN,H,RURAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [14]:
data = data.loc[(data['_merge'] == 'both'), 'doi'] = True


AttributeError: 'bool' object has no attribute 'loc'

In [13]:
len(data[data.doi == "True"])

0

In [30]:
data[data.doi == True].district.nunique()

AttributeError: 'DataFrame' object has no attribute 'doi'

In [8]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'

data = pd.read_csv(os.path.join(data_path, 'clean', 'master_data.csv'),
                  sep=",", low_memory=False)
data[data.always_eligible == True].district.nunique()

120

In [ ]:
def allyearsanddistricts(data, just_years = [2012, 2013, 2014, 2015, 2016, 2017, 2018]):
    ground_truth = {2012: 1227,
                    2013: 1228,
                    2014: 1227,
                    2015: 1219,
                    2016: 1207,
                    2017: 1203,
                    2018: 1200}
    count_correct = 0
    count_total = len(just_years)
    for yr in just_years:
        if len(data[data.year == yr]) == ground_truth[yr]:
            count_correct = count_correct + 1
        if len(data[data.year == yr]) != ground_truth[yr]:
            print('Year ', yr, 'should have', ground_truth[yr], 'districts. But has ', len(data[data.year == yr]))
    correct = (count_correct == count_total)
    return correct